In [41]:
import dash
import dash_leaflet as dl
from shapely.geometry import Point, shape
from dash import html, Output, Input, dcc, State
import osmnx as ox
import time
import json
import geopandas as gpd  
from isochrone_generator import IsochroneGenerator

####

class DashApp:
    def __init__(self):
        self.app = dash.Dash(__name__)
        self.city_boundaries = {}
        self.isochrone_dict = {}
        self.city_graphs = {}
        self.setup_layout()
        self.register_callbacks()
        self.generator = IsochroneGenerator()

    ####

    def setup_layout(self):
        self.app.layout = html.Div([
            html.Div([
                # Left side (Map)
                html.Div([
                    # Loading spinner for the map (this will show while waiting for callback)
                    dcc.Loading(
                        id="loading-isochrone", 
                        type="circle",  # You can choose from 'circle', 'dot', or 'default'
                        overlay_style={"visibility": "visible", "filter": "blur(2px)"},
                        children=dl.Map(
                            id='map',
                            center=[52.778151, -1.840227],  # Initial map center
                            zoom=5,  # Initial zoom
                            style={'width': '100%', 'height': '80vh'},  # Ensure map occupies 80% of screen height
                            children=[
                                dl.LayersControl(
                                    children=[
                                        dl.BaseLayer(dl.TileLayer(), name='Base Layer', checked=True),
                                    ]
                                )
                            ]
                        )
                    )
                ], style={'flex': 2, 'height': '100vh'}),  # Left side takes more space
    
                # Right side (Form with buttons)
                html.Div([
                    html.Div([
                        dcc.Input(id='citynameid', placeholder='London, UK', type='text', value="", style={'width': '100%'})  # City name input
                    ], style={'padding': '10px 0'}),  
    
                    html.Div([
                        html.Button("Search", id="btn", style={'width': '100%'})  # Search button
                    ], style={'padding': '10px 0'}),  
    
                    html.Div([
                        html.Button("Add boundary", id="boundarybtn", style={'width': '100%'})  # Add boundary button
                    ], style={'padding': '10px 0'}),  
                    
                    html.Div([
                        html.Div(children="Click on the Map to get Lat and Lon", id="textid", style={'width': '100%'})  
                    ], style={'padding': '10px 0'}),  
                    
                    html.Div([
                        dcc.Input(id='latid', placeholder='Enter Latitude', type='number', value="", style={'width': '27%', 'marginRight': '2%'}),
                        dcc.Input(id='lonid', placeholder='Enter Longitude', type='number', value="", style={'width': '27%', 'marginRight': '2%'}),
                        dcc.Input(id='timeid', placeholder='Enter Time Limit', type='number', value="", style={'width': '30%'})
                    ], style={'padding': '10px 0', 'display': 'flex', 'flexWrap': 'wrap'}),  # Flex to make inputs in one row
    
                    html.Div([
                        html.Button("Add isochrone", id="isochronebtn", style={'width': '100%'})  # Add isochrone button
                    ], style={'padding': '10px 0'}, title = 'An isochrone shows areas reachable within a set time from a location.'),
    
                    html.Div([
                        html.Button("Add Shortest Paths", id="shortest_path_btn", style={'width': '100%'})  # Add shortest paths button
                    ], style={'padding': '10px 0'}, title = 'Shortest paths inside the isochrone show the quickest routes to reach destinations within the time limit.'),
    
                    html.Div([
                        html.Button("Add Road Network", id="road_network_btn", style={'width': '100%'})  # Add road network button
                    ], style={'padding': '10px 0'}, title = 'The road network inside the isochrone represents all accessible roads within the defined time area.'),
    
                ], style={'padding': '20px', 'height': '100vh', 'flex': 1})  # Set width and padding for the right panel
            ], style={'display': 'flex'}),  # Use flexbox to align left and right sections
    
            # ConfirmDialog to show the message
            dcc.ConfirmDialog(
                id='confirm_dialog',
                message='',
                displayed=False  
            ),


            dcc.Store(id='layers-store', data=[]),
            
        ])

    ####

    def register_callbacks(self):
        # Callback for searching and centering the map
        @self.app.callback(
            [Output("map", "viewport"),  # Map viewport
             Output('confirm_dialog', 'displayed', allow_duplicate=True),  # Show confirm dialog
             Output('confirm_dialog', 'message', allow_duplicate=True),  # Message in confirm dialog
            ],
            Input("btn", "n_clicks"),  # Trigger on search button click
            State("citynameid", "value"),  # Get the current value of the city input
            prevent_initial_call=True
        )
        def update_map_search(n_search_clicks, city_name):
            new_center = [52.778151, -1.840227]  
            zoom = 5
            dialog_message = ""  
            show_dialog = False   
    
            # If "Search" button is clicked, center the map on the city without adding boundaries
            if n_search_clicks:

                if city_name is None or city_name == '':
                    return dash.no_update, True, 'Please enter city name'

                if not city_name in self.city_boundaries:
                    try: 
                        # If successful, get city coordinates and update the map center
                        lat, lon = self.get_city_coordinates(city_name)
                        new_center = [lat, lon]
                        zoom = 10
        
                    except ValueError as e:
                        # If an error occurs, show the error message and update the map center
                        new_center = [52.778151, -1.840227]  
                        zoom = 5
                        dialog_message = f"City not found! \nError: {e}"  
                        show_dialog = True  # Set the flag to display the dialog
                    # Return the updated map center, zoom level, and dialog state/message
                    return dict(center=new_center, zoom=zoom, transition="flyTo"), show_dialog, dialog_message
                
            return dash.no_update, show_dialog, dialog_message
            
        ####

        # Callback for adding the boundary to the map
        @self.app.callback(
            [Output('layers-store', 'data', allow_duplicate=True),
             Output('confirm_dialog', 'displayed', allow_duplicate=True),
             Output('confirm_dialog', 'message', allow_duplicate=True)], 
            Input("boundarybtn", "n_clicks"),  # Trigger when "Add boundary" is clicked
            State("citynameid", "value"),  # Get the current value of the city input
            State('layers-store', 'data'),
            prevent_initial_call=True
        )
        def update_map_boundary(n_boundary_clicks, city_name, existing_layers):
            dialog_message = ''
            show_dialog = False
            
            if n_boundary_clicks:
                if city_name is None or city_name == '':
                    return dash.no_update, True, 'Please enter a Valid city name'
                
                if not any(layer['props']['name'] == f'{city_name}_boundary' for layer in existing_layers):
                    try:
                        # Get the boundary as a GeoDataFrame
                        gdf = ox.geocode_to_gdf(city_name)
                        geojson_data = gdf.geometry.to_json()  
                        geojson_dict = json.loads(geojson_data)  
                        new_layer = dl.Overlay( dl.GeoJSON( data=geojson_dict, style={"color": "black", "weight": 2, "opacity": 1, "fillColor": "white", "fillOpacity": 0.2}
                                                    ), name=f'{city_name}_boundary', checked=True)
                        
                        existing_layers.append(new_layer)
                        self.city_boundaries[city_name] = geojson_dict
                        
                        # If city graph does not exist, generate it
                        if city_name not in self.city_graphs:
                            self.generate_graph(city_name)
                            
                    except Exception as e:
                        dialog_message = f"Error: {e}"  
                        show_dialog = True  
                        
                    return existing_layers, show_dialog, dialog_message
                        
            return dash.no_update, show_dialog, dialog_message


        # Define the callback to update the map when the button is clicked
        @self.app.callback(
            [Output('layers-store', 'data', allow_duplicate=True),
             Output('confirm_dialog', 'displayed', allow_duplicate=True),
             Output('confirm_dialog', 'message', allow_duplicate=True)],
            State('citynameid', 'value'),
            State('latid', 'value'),
            State('lonid', 'value'),
            State('timeid', 'value'),
            Input('isochronebtn', 'n_clicks'),
            State('layers-store', 'data'),
            prevent_initial_call=True,
        )
        def update_isochrone(cityname, lat, lon, time, n_clicks, existing_layers):
            # Ensure that when the button is clicked, all fields are checked
            if n_clicks > 0:
                # If any of the fields are empty, show an error message
                if not lat or not lon or not time or not cityname:
                    return None, True, 'Null or Invalid input'
        
                # Check if the values are empty strings or None
                if lat == "" or lon == "" or time == "" or cityname == "":
                    return None, True, 'Null or Invalid input'
        
                cityboundary = self.city_boundaries.get(cityname)
                polygon = shape(cityboundary['features'][0]['geometry'])

                # Check if the point is inside the polygon
                if not polygon.contains(Point(lon, lat)):
                    return None, True, 'The point is outside the city boundary!' 
                
                # If all fields are valid, proceed with isochrone generation
                if not any(layer.get('props', {}).get('id') == f'{lat}_{lon}_{time}_isochrone' for layer in existing_layers) and cityname in self.city_graphs:
                    #generator = self.city_graphs[cityname]
                    # Generate new isochrone
                    new_isochrone = self.generator.generate_isochrone(lat, lon, time)
                    
                    # Convert the isochrone to a GeoDataFrame and then to GeoJSON
                    new_isochrone_gdf = gpd.GeoDataFrame(index=[0], crs='epsg:4326', geometry=[new_isochrone])
                    new_geojson_data = new_isochrone_gdf.geometry.to_json()
        
                    # Return the new GeoJSON data to the map
                    new_layer = dl.Overlay(dl.GeoJSON( data=json.loads(new_geojson_data), style={"color": "red", "weight": 2, "opacity": 1, "fillColor": "white", "fillOpacity": 0.2}
                                                    ), name='isochrone', id = f'{lat}_{lon}_{time}_isochrone', checked=True)
                        
                    existing_layers.append(new_layer)

                    new_layer = dl.Overlay(dl.Marker(position=[lat, lon]), name = 'Marker', id=f'{lat}_{lon}_marker', checked = True)
                    existing_layers.append(new_layer)
                 
                    return existing_layers, False, "" 
        
            # Return empty GeoJSON if the button hasn't been clicked yet
            return existing_layers, False, ""
        
        ####

        # Define the callback to update the latitude and longitude when the map is clicked
        @self.app.callback(
            [Output('confirm_dialog', 'displayed', allow_duplicate=True),
             Output('confirm_dialog', 'message', allow_duplicate=True),
             Output('latid', 'value'),
             Output('lonid', 'value')
            ],
            [State('citynameid', 'value'),  
             Input('map', 'clickData')
            ], 
             prevent_initial_call=True, # Listen for clicks on the map
        )
        def update_lat_lon(cityname, clickData):
  
            if clickData:
                # Extract the latitude and longitude from the clickData
                lat = clickData['latlng']['lat']
                lon = clickData['latlng']['lng']
                
                # Create a Point object from the latitude and longitude
                point = Point(lon, lat)

                if not cityname in self.city_boundaries:
                    return False, '', None, None
                # Get the boundary of the selected city
                cityboundary = self.city_boundaries.get(cityname)

                # Convert the GeoJSON to a Shapely Polygon
                polygon = shape(cityboundary['features'][0]['geometry'])
                    
                # Check if the point is inside the polygon
                if polygon.contains(point):
                    return False, '', lat, lon
                else:
                    return True, 'The point is outside the city boundary!', None, None, existing_layers
            # Return None, None if no click or city boundary is not found
            return False, '', None, None

         ####
        @self.app.callback(
            Output('layers-store', 'data', allow_duplicate=True),
            Input('shortest_path_btn', 'n_clicks'),
            State('layers-store', 'data'),
            State('latid', 'value'),
            State('lonid', 'value'),
            State('timeid', 'value'),
            prevent_initial_call=True
        )
        def add_shortest_path(n_clicks, existing_layers, lat, lon, time):
            if n_clicks is not None and n_clicks > 0:
                if not any(layer.get('props', {}).get('id') == f'{lat}_{lon}_{time}_shortest_path' for layer in existing_layers):
                    if any(layer.get('props', {}).get('id') == f'{lat}_{lon}_{time}_isochrone' for layer in existing_layers):
                        shortest_paths = self.generator.generate_shortest_paths()
                        new_layer = dl.Overlay(dl.LayerGroup(shortest_paths), name='shortest_path', id=f'{lat}_{lon}_{time}_shortest_path', checked = True)
                        existing_layers.append(new_layer)
            return existing_layers

        ####
        @self.app.callback(
            Output('layers-store', 'data', allow_duplicate=True),
            Input('road_network_btn', 'n_clicks'),
            State('layers-store', 'data'),
            State('latid', 'value'),
            State('lonid', 'value'),
            State('timeid', 'value'),
            prevent_initial_call=True
        )
        def add_road_network(n_clicks, existing_layers, lat, lon, time):
            if n_clicks is not None and n_clicks > 0:
                if not any(layer.get('props', {}).get('id') == f'{lat}_{lon}_{time}_network' for layer in existing_layers):
                    if any(layer.get('props', {}).get('id') == f'{lat}_{lon}_{time}_isochrone' for layer in existing_layers):
                        road_network = self.generator.graph_to_geojson()
                        new_layer = dl.Overlay(
                            dl.GeoJSON(data=road_network), name = 'road_network', id = f'{lat}_{lon}_{time}_network', checked = True
                        )
                        existing_layers.append(new_layer)    
            return existing_layers
            
        # Callback to update the map layers based on the store
        @self.app.callback(
            Output('map', 'children'),
            Input('layers-store', 'data'),
            prevent_initial_call=True
        )
        def update_map_layers(layers):
            base_layer = dl.BaseLayer(dl.TileLayer(), name='base', checked=True)
            layers_control = dl.LayersControl(
                children=[base_layer] + layers  
            )
            return [layers_control]  
                  
    # Generate graph
    def generate_graph(self, city_name: str):
        if city_name not in self.city_graphs:
            try:
                if city_name == 'Somerset, UK':
                    graph = self.generator._load_graph_from_file('Somerset_UK_drive.graphml')
                else:
                    # Attempt to create a graph for the city
                    graph = self.generator._load_graph_from_place(place_name=city_name)
                
                self.city_graphs[city_name] = graph
                return self.city_graphs[city_name]  
                
            except Exception as e:
                # If an error occurs, handle it and return the error message
                error_message = f"Failed to create graph for {city_name}. Error: {str(e)}"
                return error_message  # Return the error message instead of raising an exception
                
        # If the graph already exists in the dictionary, return it directly
        return self.city_graphs.get(city_name)        

    ####

    def get_city_coordinates(self, city_name):
        """Get coordinates for the city."""
        try:
            gdf = ox.geocode_to_gdf(city_name)
            lat, lon = gdf.geometry.centroid.y.values[0], gdf.geometry.centroid.x.values[0]
            return lat, lon
        
        except Exception as e:
            return f"Error: {e}"
            
# Run the app
if __name__ == '__main__':
    app_instance = DashApp()
    app_instance.app.run_server(debug=True)